## Analyzing Subway Data

** Q1. Exploratory Data Analysis **

Let's examine the hourly entries in our NYC subway data and determine what distribution the data follows.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def entries_histogram(turnstile_weather) :
    plt.figure()
    turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain'] ==1].hist() 
    turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain'] ==0].hist() 
    return plt

**Q2. Does entries data from the previous exercise seem normally distributed? **

** And can we run Welch's T test on entries data? **

-> No. It's not normally distributed.

-> We cannot run Welch's T test. T test is based on data from normally distributed samples.

-> So we do the Mann-Whitney U-Test (or bootstrapping)

**Q3. Mann-Whitney U-Test **

Your function should return

    1) the mean of entries with rain
    2) the mean of entries without rain
    3) the Mann-Whitney U-statistic and p-value comparing the number of entries with rain and the number of entries without rain
    
http://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html

In [4]:
import numpy as np
import scipy
import scipy.stats
import pandas

def mann_whitney_plus_means(turnstile_weather):

    with_rain_mean = turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain']==1].mean()
    without_rain_mean = turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain']==0].mean()
    Mann_Whitney = scipy.stats.mannwhitneyu(turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain']==1], 
    turnstile_weather['ENTRIESn_hourly'][turnstile_weather['rain']==0], 
    use_continuity = True, alternative = 'greater')
    # alternative가 greater인 것에 주의! 여기서는 비오는날에 지하철 타는 사람이 더 많은지를 확인하고 싶은 것.
    
    U = Mann_Whitney[0]
    p = Mann_Whitney[1]
    
    return with_rain_mean, without_rain_mean, U, p # leave this line for the grader


**Q4. Was the test significant?**

Yes, we can say there is a difference between two data.

**Q5. Linear Regression**

You need to

    1) implement the compute_cost() and gradient_descent() procedures
    2) Select features (in the predictions procedure) and make predictions
    

In [ ]:
import numpy as np
import pandas
from ggplot import *

"""
In this question, you need to:
1) implement the compute_cost() and gradient_descent() procedures
2) Select features (in the predictions procedure) and make predictions.

"""

def normalize_features(df):
    """
    Normalize the features in the data set.
    """
    mu = df.mean()
    sigma = df.std()
    
    if (sigma == 0).any():
        raise Exception("One or more features had the same value for all samples, and thus could " + \
                         "not be normalized. Please do not include features with only a single value " + \
                         "in your model.")
    df_normalized = (df - df.mean()) / df.std()

    return df_normalized, mu, sigma

def compute_cost(features, values, theta):
    """
    Compute the cost function given a set of features / values, 
    and the values for our thetas.
    
    This can be the same code as the compute_cost function in the lesson #3 exercises,
    but feel free to implement your own.
    """
    m = len(values)
    sum_of_square_error = np.square(np.dot(features, theta) - values).sum()
    cost = sum_of_square_error / (2*m)
    # your code here

    return cost

def gradient_descent(features, values, theta, alpha, num_iterations):
    """
    Perform gradient descent given a data set with an arbitrary number of features.
    
    This can be the same gradient descent code as in the lesson #3 exercises,
    but feel free to implement your own.
    """
    
    m = len(values)
    cost_history = []

    for i in range(num_iterations):
        predicted_values = np.dot(features, theta)
        theta = theta-alpha/m *np.dot( (predicted_values-values), features)
        cost_history.append(compute_cost(features, values, theta))
    return theta, pandas.Series(cost_history)

def predictions(dataframe):

    # Select Features (try different features!)
    features = dataframe[['rain', 'precipi', 'Hour', 'meantempi']]
    
    # Add UNIT to features using dummy variables
    dummy_units = pandas.get_dummies(dataframe['UNIT'], prefix='unit')
    features = features.join(dummy_units)
    
    # Values
    values = dataframe['ENTRIESn_hourly']
    m = len(values)

    features, mu, sigma = normalize_features(features)
    features['ones'] = np.ones(m) # Add a column of 1s (y intercept)
    
    # Convert features and values to numpy arrays
    features_array = np.array(features)
    values_array = np.array(values)

    # Set values for alpha, number of iterations.
    alpha = 0.1 # please feel free to change this value
    num_iterations = 75 # please feel free to change this value

    # Initialize theta, perform gradient descent
    theta_gradient_descent = np.zeros(len(features.columns))
    theta_gradient_descent, cost_history = gradient_descent(features_array, 
                                                            values_array, 
                                                            theta_gradient_descent, 
                                                            alpha, 
                                                            num_iterations)
    
    plot = None
    # -------------------------------------------------
    # Uncomment the next line to see your cost history
    # -------------------------------------------------
    # plot = plot_cost_history(alpha, cost_history)
    # 
    # Please note, there is a possibility that plotting
    # this in addition to your calculation will exceed 
    # the 30 second limit on the compute servers.
    
    predictions = np.dot(features_array, theta_gradient_descent)
    return predictions, plot


def plot_cost_history(alpha, cost_history):
   """This function is for viewing the plot of your cost history.
   You can run it by uncommenting this

       plot_cost_history(alpha, cost_history) 

   call in predictions.
   
   If you want to run this locally, you should print the return value
   from this function.
   """
   cost_df = pandas.DataFrame({
      'Cost_History': cost_history,
      'Iteration': range(len(cost_history))
   })
   return ggplot(cost_df, aes('Iteration', 'Cost_History')) + \
      geom_point() + ggtitle('Cost History for alpha = %.3f' % alpha )




**Q6. Plotting Residuals **

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt

def plot_residuals(turnstile_weather, predictions):
    '''
    Using the same methods that we used to plot a histogram of entries
    per hour for our data, why don't you make a histogram of the residuals
    (that is, the difference between the original hourly entry data and the predicted values).
    Try different binwidths for your histogram.

    Based on this residual histogram, do you have any insight into how our model
    performed?  Reading a bit on this webpage might be useful:

    http://www.itl.nist.gov/div898/handbook/pri/section2/pri24.htm
    '''
    
    plt.figure()
    (turnstile_weather['ENTRIESn_hourly'] - predictions).hist()
    return plt

**Q7. Compute R^2**

In [6]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import sys

def compute_r_squared(data, predictions):
    '''
    In exercise 5, we calculated the R^2 value for you. But why don't you try and
    and calculate the R^2 value yourself.
    
    Given a list of original data points, and also a list of predicted data points,
    write a function that will compute and return the coefficient of determination (R^2)
    for this data.  numpy.mean() and numpy.sum() might both be useful here, but
    not necessary.

    Documentation about numpy.mean() and numpy.sum() below:
    http://docs.scipy.org/doc/numpy/reference/generated/numpy.mean.html
    http://docs.scipy.org/doc/numpy/reference/generated/numpy.sum.html
    '''
    
    # your code here
    SST = ((data-np.mean(data))**2).sum()
    SSReg = ((predictions-data)**2).sum()
    r_squared = 1 - SSReg / SST
    return r_squared



** Q8. More Linear Regressions **